In [ ]:
import numpy as np
import control as con
from numpy import linalg as LA

import cvxpy
import optim_tools #own file with helper
import boris_tools

import matplotlib.pyplot as plt
%matplotlib inline

In [ ]:
print cvxpy.installed_solvers()

In [ ]:
#############################
# Roboter pt2 (ohne delay)  #
#############################

A = np.matrix([[  0.,         -55.99932527],
               [ 10.,         -43.64272128]])

b = np.matrix([[-5.58731344],
               [ 0.        ]])
B = b

c = np.matrix([ 0.,  -10.])
C = c

d = np.matrix([0])
D = d

u_max = 0.5
U_max = [u_max]

n = len(b)

delay = 0.032

X00 = [np.matrix([-0.5, -0.025]).T,
       np.matrix([-0.5,  0.025]).T,
       np.matrix([ 0.5, -0.025]).T,
       np.matrix([ 0.5,  0.025]).T,
      ]

X0 = X00

In [ ]:
def Tri(i, n_u):
    if n_u>1:
        print "not implemented"
        print "alle permutationen mit einsen auf der hauptdiagonalen!"
        raise Error()
        
    if i==1:
        return np.eye(n_u)
    else:
        # only true for n_u == 1
        return np.zeros((n_u, n_u))

def negTri(i, n_u):
    return np.eye(n_u) - Tri(i, n_u)

print Tri(1,1)
print Tri(2,1)
print
print negTri(1,1)
print negTri(2,1)


In [ ]:
%%time
# Satz 6.6

# Init
n = B.shape[0] # get dim of system
n_u = B.shape[1] 

# Variables
X = cvxpy.Semidef(n)
Y = cvxpy.Semidef(n)
W = cvxpy.Variable(n_u)

Ak_h = cvxpy.Semidef(*A.shape)
Bk_h = cvxpy.Variable(*B.shape)
Ck_h = cvxpy.Variable(*C.shape)
Dk_h = cvxpy.Variable(*D.shape)

Ch = cvxpy.Variable(*C.shape)
Dh = cvxpy.Variable(*D.shape)

# Substitutionen
C_hat = lambda i: Tri(i, n_u)*Ck_h + negTri(i, n_u)*Ch
D_hat = lambda i: Tri(i, n_u)*Dk_h + negTri(i, n_u)*Dh

Xi = cvxpy.bmat([[ A*X + B*Ck_h, A + B*Dk_h*C ],
                 [ Ak_h,         Y*A + Bk_h*C ]])

I = np.eye(n)

# Bisection parameter
g = cvxpy.Parameter(sign='positive')

# Pole restriction
ro = cvxpy.Parameter(sign='positive') # Real <=-ro
ni = cvxpy.Parameter(sign='positive') # |Imag| <= ni*Real

ro.value = 30
ni.value = 2

# Define Constraints

# (6.39a)
const_639a = cvxpy.bmat([[X, I],
                         [I, Y]]) == cvxpy.Semidef(2*n)



# (6.39b)
const_639b = cvxpy.bmat([[ X*A.T + A*X + B*Ck_h + (B*Ck_h).T, Ak_h.T + A + B*Dk_h*C            ],
                         [ cvxpy.Variable(n, n),          A.T*Y + Y*A + Bk_h*C + (Bk_h*C).T]]) + \
             2*g*cvxpy.bmat([[X, I],
                             [I, Y]]) == -cvxpy.Semidef(2*n)
    
# (6.39c)
const_639c = [cvxpy.bmat([[ X*A.T + A*X + B*C_hat(i) + (B*C_hat(i)).T, Ak_h.T + A + B*D_hat(i)*C            ],
                          [ cvxpy.Variable(n, n),                      A.T*Y + Y*A + Bk_h*C + (Bk_h*C).T]]) == -cvxpy.Semidef(2*n) for i in range(2, (2**n_u)+1)]
    
    
# (6.39d)
const_639d = cvxpy.bmat([[ X,  I,    Ch.T     ],
                         [ I,  Y,    (Dh*C).T ],
                         [ Ch, Dh*C, W        ]]) == cvxpy.Semidef(2*n+n_u)

const_639e = [W[j,j] <= U_max[j]**2 for j in range(0, n_u)]

const_639f = [ X0[k].T*Y*X0[k] <= 1.0
                            for k in range(0, len(X0))]

const_621a = Xi.T + Xi + ro*cvxpy.bmat([[X, I],
                                        [I, Y]]) == cvxpy.Semidef(2*n)

const_621b = cvxpy.bmat([[ ni*(Xi.T + Xi), Xi.T - Xi ],
                         [ -Xi.T + Xi,     ni*(Xi.T + Xi) ]]) == -cvxpy.Semidef(4*n)

# Collect all constraints
constraints_639 = []
constraints_639.append(const_639a)
constraints_639.append(const_639b)
constraints_639.extend(const_639c)
constraints_639.append(const_639d)
constraints_639.extend(const_639e)
constraints_639.extend(const_639f)

constraints_639.append(const_621a)
constraints_639.append(const_621b)


# Form problem.
prob_66 = cvxpy.Problem(cvxpy.Minimize(0), constraints_639)

# bisection
[[X_o, Y_o, W_o,
  Ak_h_o, Bk_h_o, Ck_h_o, Dk_h_o,
  Ch_o, Dh_o], g_o] = optim_tools.bisect_max(0, None, prob_66, g, [X, Y, W, Ak_h, Bk_h, Ck_h, Dk_h, Ch, Dh], bisect_verbose=True,
                                                      bisection_tol=0.01,
                                                      #solver=cvxpy.CVXOPT, verbose=False)
                                                      solver=cvxpy.MOSEK, verbose=False)
                                                      #solver=cvxpy.SCS, max_iters=50000)

print "g:", g_o

print
print '-----------------------'

In [ ]:
# QR Decomposition for M and N
M, NT = LA.qr(I - X_o.dot(Y_o))
N = NT.T
assert(np.allclose(I - X_o.dot(Y_o), M.dot(N.T)))


# Reconstruction
Ek = -LA.solve(N, Y_o*B)
Dk = np.matrix(Dk_h_o)

Ck = LA.solve(M, (Ck_h_o - Dk.dot(C).dot(X_o)).T).T
#Ck_1 = (Ck_h_o - Dk.dot(C).dot(X_o)).dot(LA.inv(M).T) #Check
#assert(np.allclose(Ck_1, Ck))

Bk = LA.solve(N, Bk_h_o)

temp_alpha = LA.solve(N, (Ak_h_o-Y_o.dot(A).dot(X_o)))
temp_beta = Bk.dot(C).dot(X_o)
Ak = (LA.solve(M, temp_alpha.T) - LA.solve(M, temp_beta.T)).T

#Ak_1 = LA.solve(N, (Ak_h_o-Y_o.dot(A).dot(X_o))).dot(LA.inv(M).T) - Bk.dot(C).dot(X_o).dot(LA.inv(M).T) #Check
#assert(np.allclose(Ak_1, Ak))


In [ ]:
from numpy.linalg import solve

def sat(v, u_max):
    return np.clip(v, -u_max, u_max)

def control_func(y, s, x, k, A, b, c):
    #v = -np.linalg.inv(c.dot(np.linalg.inv(A-b.dot(k.T))).dot(b))
    #u = v.dot(s)-k.T.dot(x)
    N = -c.dot(solve(A-b.dot(k.T), b))
    u = solve(N, np.array([s]))-k.T.dot(x)
    return u

class control_func_dyn_out():
    def __init__(self, Ak, Bk, Ck, Dk, Ek, umax=None, dT=1e-3):
        self.Ak = Ak
        self.Bk = Bk
        self.Ck = Ck
        self.Dk = Dk
        self.Ek = Ek
        self.umax = umax
        self.dT = dT

        self.z = np.zeros((len(b),1))
        #print "init:", self.z
        
    def estimate(self, y, u):
        if self.umax is not None:
            u = optim_tools.sat(u, self.umax)
        
        z_dot = self.Ak.dot(self.z) + self.Bk.dot(y) + self.Ek.dot(u)
        #print "u", u
        #print "Ek", self.Ek
        #print "Ek*u", self.Ek.dot(u)
        return self.z + z_dot*self.dT
        
    def regulate(self, y, s, x):
        u = self.Ck.dot(self.z) + self.Dk.dot(y)
        #print self.Dk
        #print y
        #print self.Dk.dot(y)
        # Saturate 
        if self.umax is not None:
            u = optim_tools.sat(u, self.umax)
            
        self.z = self.estimate(y, u)
        #print "esti:", self.z
        
        return u

T = np.arange(0, 0.4, 1e-3) 

#s: input, e.g., step function with amplitude of 0.2
s = np.zeros(len(T));
#s = np.ones(len(T))*0.1;


# Initial state
#x0 = np.zeros((len(b),1))
x0 = np.ones((len(b),1))*0.1

T[1]-T[0]

In [ ]:
import collections
import math

''' State Space Simulator '''
def simulate(A, B, C, D, regulator_func, s, T, delay=None, umax=None, x0=0):
    #intitialize y, u
    y = np.matrix(np.zeros((C.shape[0],len(T))))
    u = np.zeros((len(T),np.size(x0,1)))
    u_sat = np.zeros((len(T),np.size(x0,1)))
    if type(x0) is int:
        xt = np.matrix([x0]*len(A)).T
        print "x0 = \n{}".format(xt)
    else:
        xt = x0

    if delay:
        s_queue = collections.deque(maxlen=int(math.ceil(delay/(T[1]-T[0]))))
        
    for i, t in enumerate(T):
        if delay:
            s_queue.append(s[i])
            if len(s_queue) == int(math.ceil(delay/(T[1]-T[0]))):
                s_delay = s_queue[0]
            else:
                s_delay = 0
        else:
            s_delay = s[i]
        
        #print "----------------------------", i
        #print "SS: x:", x_delay
        #print "SS: y:", y[:,i]
        u[[i],:] = regulator_func(y[:,i], s_delay, xt)

        #if i >= 2:
        #    raise Exception
        
        if umax is not None:
            u_sat[[i],:] = sat(u[[i],:], umax)
        else:
            u_sat[[i],:] = u[[i],:]

        x_dot = A.dot(xt) + B.dot(u_sat[[i],:])
        #print "SS: x_dot:", x_dot
        
        y[:,i] = C.dot(xt) + D.dot(u_sat[[i],:])

        if i < len(T)-1:
            xt = xt + x_dot*(T[i+1]-T[i])
            #print "x_dot (SS)", x_dot
    return y, u, u_sat

In [ ]:
from functools import partial

y0, u0, u0_sat = simulate(A, b, c, d, 
                                       partial(optim_tools.openLoop),
                                       s, T, delay=delay, umax=u_max, x0=np.ones((len(b),1))*0.1)

y1, u1, u1_sat = simulate(A, b, c, d, 
                                   control_func_dyn_out(Ak, Bk, Ck, Dk, Ek, umax=u_max).regulate,
                                   s, T, delay=delay, umax=u_max, x0=np.ones((len(b),1))*0.1)


In [ ]:
%pylab inline
pylab.rcParams['figure.figsize'] = (10, 6)
import matplotlib.pyplot as plt

line0, = plt.plot(T[:], np.array(y1[0,:].T), 'r', label='dyn_out')
line0, = plt.plot(T[:], np.array(y0[0,:].T), 'g--', label='open')


#first_legend = plt.legend(handles=[line1], loc=1)
plt.legend(bbox_to_anchor=(1.05, 1), loc=2, borderaxespad=0.)
plt.xlabel('T')
plt.ylabel('y')
plt.title('Closed Loop Step Response')
plt.show()


line0, = plt.plot(T, u1, 'b--', label='u1')
line2, = plt.plot(T, u1_sat, 'b', label='u1_sat')



#>first_legend = plt.legend(handles=[line1, line2, line1b, line2b], loc=1)
plt.legend(bbox_to_anchor=(1.05, 1), loc=2, borderaxespad=0.)
plt.xlabel('T')
plt.ylabel('u')
plt.title('Output values')
plt.show()
